<a href="https://colab.research.google.com/github/anshupandey/ms-generativeai-apr2025/blob/main/code10_AI_Agents_with_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Understanding LangChain Agents and Chains

## Chains

Chains refer to sequences of calls - whether to an LLM, a tool, or a data preprocessing step. The primary supported way to do this is with LCEL.

## Agents

Agents can be thought of as the chain responsible for deciding what step to take next. This is usually powered by a language model, a prompt, and an output parser.

Different agents have different prompting styles for reasoning, different ways of encoding inputs, and different ways of parsing the output. For a full list of built-in agents see agent types. You can also easily b


## Chains v/s Agents

The core idea of agents is to use a language model to choose a sequence of actions to take. In chains, a sequence of actions is hardcoded (in code). In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.




In [ ]:
# set environment variable: TAVILY_API_KEY=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

In [1]:
# Installing necessary libraries
!pip install langchain-openai langchain langchain-core langchain-community --quiet
!pip install httpx langchain-experimental --quiet
!pip install openai google-search-results tiktoken wikipedia pyowm --quiet

In [2]:
import os
os.environ["OPENWEATHERMAP_API_KEY"] = os.environ['OWM']
model_name="gpt-4o"

# LangChain AI Agents

The core idea of agents is to use a language model to choose a sequence of actions to take. In chains, a sequence of actions is hardcoded (in code). In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.



In [3]:
# Importing necessary modules for the agent
from langchain.agents import load_tools      # Imports the load_tools function for loading various tools in the langchain package
from langchain_openai import ChatOpenAI # Imports the ChatOpenAI class to interact with OpenAI's models

In [27]:
# Initializing a Language Model (LLM) instance
model = ChatOpenAI(model=model_name,
                temperature=0,seed=1)

In [19]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)


[{'title': 'Weather in San Francisco', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1743743565, 'localtime': '2025-04-03 22:12'}, 'current': {'last_updated_epoch': 1743742800, 'last_updated': '2025-04-03 22:00', 'temp_c': 12.8, 'temp_f': 55.0, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 5.4, 'wind_kph': 8.6, 'wind_degree': 284, 'wind_dir': 'WNW', 'pressure_mb': 1018.0, 'pressure_in': 30.06, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 62, 'cloud': 0, 'feelslike_c': 12.2, 'feelslike_f': 53.9, 'windchill_c': 11.1, 'windchill_f': 52.0, 'heatindex_c': 11.7, 'heatindex_f': 53.0, 'dewpoint_c': 8.8, 'dewpoint_f': 47.9, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 0.0, 'gust_mph': 8.8, 'gust_kph': 14.2}}", 'score': 0.95068

In [20]:
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

In [21]:
print(search.name)
print(search.description)
print(search.args)

tavily_search_results_json
A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.
{'query': {'description': 'search query to look up', 'title': 'Query', 'type': 'string'}}


In [28]:
model_with_tools = model.bind_tools(tools)

In [29]:
from langchain_core.messages import HumanMessage

response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


In [30]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_w4kGHTnm84hE3Z7injpwJPUt', 'type': 'tool_call'}]


### Implementing Agent with LangChain

In [31]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("human", "{input}"),
        # Placeholders fill up a **list** of messages
        ("placeholder", "{agent_scratchpad}"),
    ]
)
prompt

ChatPromptTemplate(input_variables=['input'], optional_variables=['agent_scratchpad'], input_types={'agent_scratchpad': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessag

In [32]:
from langchain.agents import create_tool_calling_agent
agent = create_tool_calling_agent(model, tools, prompt)

In [33]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [34]:
agent_executor.invoke({"input": "hi!"})



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


{'input': 'hi!', 'output': 'Hello! How can I assist you today?'}

In [35]:
agent_executor.invoke({"input": "What's the weather in Mumbai?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'current weather in Mumbai'}`


[{'title': 'Weather Mumbai in April 2025: Temperature & Climate', 'url': 'https://en.climate-data.org/asia/india/maharashtra/mumbai-29/t/april-4/', 'content': 'Are you planning a holiday with hopefully nice weather in Mumbai in April 2025? ... 4. April, 27 °C | 81 °F, 28 °C | 82 °F, 26 °C | 78 °F, 28 °C | 83 °F, 5.8 mm', 'score': 0.9345448}, {'title': 'Weather in Mumbai', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Mumbai', 'region': 'Maharashtra', 'country': 'India', 'lat': 18.975, 'lon': 72.826, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1743744336, 'localtime': '2025-04-04 10:55'}, 'current': {'last_updated_epoch': 1743743700, 'last_updated': '2025-04-04 10:45', 'temp_c': 31.0, 'temp_f': 87.8, 'is_day': 1, 'condition': {'text': 'Mist', 'icon': '//cdn.weatherapi.com/weather/64x64/day/143.png', 'code': 1030}, 'wind_mph': 3.8, 'wi

{'input': "What's the weather in Mumbai?",
 'output': 'The current weather in Mumbai is as follows:\n\n- **Temperature**: 31.0°C (87.8°F)\n- **Condition**: Mist\n- **Wind**: 3.8 mph (6.1 kph) from the northwest\n- **Humidity**: 66%\n- **Cloud Cover**: 50%\n- **Feels Like**: 38.6°C (101.5°F)\n- **Visibility**: 3.0 km (1.0 mile)\n- **UV Index**: 5.7\n\nThe weather is misty with moderate humidity and a gentle breeze.'}

In [36]:
agent_executor.invoke({"input": "what is capital of Maldives and what's the weather there?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'capital of Maldives'}`


[{'title': 'Capital of Maldives - Maldives Capital City | Visitor Guide', 'url': 'https://www.budgetmaldives.com/maldives/maldives-capital/', 'content': "The Maldives capital is Male'. As the highest populated island, Male' is urbanized and is often regarded as a concrete jungle due to the buildings in the large", 'score': 0.902687}, {'title': 'Malé - Wikipedia', 'url': 'https://en.wikipedia.org/wiki/Mal%C3%A9', 'content': "Malé[a] is the capital and most populous city of the Maldives. With a population of 211,908 in 2022[1] within its administrative area and coterminous geographical area of 8.30 square kilometres (3.20\xa0sq\xa0mi), Malé is also one of the most densely populated cities in the world.[2][3] The city is geographically located in the southern edge of North Malé Atoll (Kaafu Atoll).[4] [...] AlifDhaandhooDhevvadhooGemanafushiKanduhulhudhooKolamaafushiKo

{'input': "what is capital of Maldives and what's the weather there?",
 'output': 'The capital of the Maldives is Malé. \n\nAs for the current weather in Malé, Maldives, it is partly cloudy with a temperature of 28.9°C (83.9°F). The wind is blowing from the north at 9.2 mph (14.8 kph), and the humidity is at 70%. The weather feels like 32.6°C (90.7°F) due to the humidity.'}

## Implementing a Multitool Agent

In [37]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import create_tool_calling_agent, AgentExecutor


prompt = ChatPromptTemplate.from_messages(
    [   ("system", "You are a helpful assistant"),
        ("human", "{input}"),
        # Placeholders fill up a **list** of messages
        ("placeholder", "{agent_scratchpad}"), ])

search = TavilySearchResults(max_results=2)
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
# Create Image Gentation model Object

tools = [search, wikipedia]

agent = create_tool_calling_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [38]:
agent_executor.invoke({"input": "Hi, How are you?"})



> Entering new AgentExecutor chain...
Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you with any questions or information you need. How can I assist you today?

> Finished chain.


{'input': 'Hi, How are you?',
 'output': "Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you with any questions or information you need. How can I assist you today?"}

In [39]:
agent_executor.invoke({"input": "What's the current weather in Dubai?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'current weather in Dubai'}`


[{'title': 'Weather in Dubai', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Dubai', 'region': 'Dubai', 'country': 'United Arab Emirates', 'lat': 25.2522, 'lon': 55.28, 'tz_id': 'Asia/Dubai', 'localtime_epoch': 1743744871, 'localtime': '2025-04-04 09:34'}, 'current': {'last_updated_epoch': 1743744600, 'last_updated': '2025-04-04 09:30', 'temp_c': 32.3, 'temp_f': 90.1, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 13.6, 'wind_kph': 22.0, 'wind_degree': 198, 'wind_dir': 'SSW', 'pressure_mb': 1012.0, 'pressure_in': 29.88, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 16, 'cloud': 0, 'feelslike_c': 31.3, 'feelslike_f': 88.4, 'windchill_c': 29.7, 'windchill_f': 85.4, 'heatindex_c': 28.4, 'heatindex_f': 83.1, 'dewpoint_c': 9.5, 'dewpoint_f': 49.0, 'vis_km':

{'input': "What's the current weather in Dubai?",
 'output': 'The current weather in Dubai is sunny with a temperature of 32.3°C (90.1°F). The wind is blowing from the south-southwest at 13.6 mph (22.0 kph), and the humidity is at 16%. There is no precipitation, and the visibility is 10 km (6 miles). The UV index is 3.6, indicating moderate exposure.'}

In [40]:
agent_executor.invoke({"input": "what is total area of USA"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'total area of the United States'}`


Page: List of U.S. states and territories by area
Summary: This is a complete list of all 50 U.S. states, its federal district (Washington, D.C.) and its major territories ordered by total area, land area and water area. The water area includes inland waters, coastal waters, the Great Lakes and territorial waters. Glaciers and intermittent bodies of water are counted as land area.

Page: List of largest lakes of the United States by area
Summary: The following is a list of the 100 largest lakes of the United States by normal surface area.  The top twenty lakes in size are as listed by the National Atlas of the United States, a publication of the United States Department of the Interior.  The area given is the normal or average area of the lake. The area of some lakes fluctuates substantially.  For those lakes partially in Canada or Mexico the area given for the lake is t

{'input': 'what is total area of USA',
 'output': 'The total area of the United States is approximately 3,809,525 square miles, which includes 3,532,316 square miles of land and 277,209 square miles of water.'}

## Thank You